In [3]:
import requests
from zipfile import ZipFile
from io import BytesIO

url = 'https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/lateral_movement/host/covenant_copy_smb_CreateRequest.zip'
zipFileRequest = requests.get(url)
zipFile = ZipFile(BytesIO(zipFileRequest.content))
datasetJSONPath = zipFile.extract(zipFile.namelist()[0])

In [4]:
import pandas as pd
from pandas.io import json

df = json.read_json(path_or_buf=datasetJSONPath, lines=True)

C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [5]:
(
df[['@timestamp','Hostname','ShareName','SubjectUserName','SubjectLogonId','AccessMask']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 5140)
    & (~df['ShareName'].str.contains('.*IPC.*', regex=True, na=True))
    & (~df['SubjectUserName'].str.endswith('$', na=False))
]
.head()
)

,@timestamp,Hostname,ShareName,SubjectUserName,SubjectLogonId,AccessMask
58,2020-09-22T18:53:32.341Z,WORKSTATION6.theshire.local,\\*\C$,pgustavo,0x4e13b2e,0x1


In [11]:
IPCShareDf = (
# (
df[['@timestamp','Hostname','ShareName','SubjectUserName','SubjectLogonId','AccessMask']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 5140)
    & (df['ShareName'].str.contains('.*IPC.*', regex=True))
    & (~df['SubjectUserName'].str.endswith('$', na=False))
]
)

CShareDf = (
# (
df[['@timestamp','Hostname','ShareName','SubjectUserName','SubjectLogonId','AccessMask']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 5140)
    & (df['ShareName'].str.endswith('\\C$', na=False))
    & (~df['SubjectUserName'].str.endswith('$', na=False))
]
)

(
pd.merge(IPCShareDf, CShareDf,
    on = 'SubjectLogonId', how = 'inner')
)

,@timestamp_x,Hostname_x,ShareName_x,SubjectUserName_x,SubjectLogonId,AccessMask_x,@timestamp_y,Hostname_y,ShareName_y,SubjectUserName_y,AccessMask_y
0,2020-09-22T18:53:32.341Z,WORKSTATION6.theshire.local,\\*\IPC$,pgustavo,0x4e13b2e,0x1,2020-09-22T18:53:32.341Z,WORKSTATION6.theshire.local,\\*\C$,pgustavo,0x1


In [5]:
(
df[['@timestamp','Hostname','ShareName','SubjectUserName','SubjectLogonId','IpAddress','IpPort','RelativeTargetName']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 5145)
    & (df['AccessMask'] == '0x2')
    & (df['ShareName'].str.endswith('\\C$', na=False))
    & (~df['SubjectUserName'].str.endswith('$', na=False))
]
)

,@timestamp,Hostname,ShareName,SubjectUserName,SubjectLogonId,IpAddress,IpPort,RelativeTargetName
61,2020-09-22T18:53:32.342Z,WORKSTATION6.theshire.local,\\*\C$,pgustavo,0x4e13b2e,172.18.39.5,62782.0,ProgramData\GruntHTTP.exe


In [15]:
IPCShareDf = (
# (
df[['@timestamp','Hostname','ShareName','SubjectUserName','SubjectLogonId','AccessMask']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 5140)
    & (df['ShareName'].str.contains('.*IPC.*', regex=True))
    & (~df['SubjectUserName'].str.endswith('$', na=False))
]
)

CShareDf = (
# (
df[['@timestamp','Hostname','ShareName','SubjectUserName','SubjectLogonId','AccessMask']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 5140)
    & (df['ShareName'].str.endswith('\\C$', na=False))
]
)


firstJoinDf = (
pd.merge(IPCShareDf, CShareDf,
    on = 'SubjectLogonId', how = 'inner')
)

fileAccessedDf = (
# (
df[['@timestamp','Hostname','ShareName','SubjectUserName','SubjectLogonId','IpAddress','IpPort','RelativeTargetName']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 5145)
    & (df['AccessMask'] == '0x2')
    & (df['ShareName'].str.endswith('\\C$', na=False))
]
)

(
pd.merge(firstJoinDf, fileAccessedDf,
    on = 'SubjectLogonId', how = 'inner')
)

,@timestamp_x,Hostname_x,ShareName_x,SubjectUserName_x,SubjectLogonId,AccessMask_x,@timestamp_y,Hostname_y,ShareName_y,SubjectUserName_y,AccessMask_y,@timestamp,Hostname,ShareName,SubjectUserName,IpAddress,IpPort,RelativeTargetName
0,2020-09-22T18:53:32.341Z,WORKSTATION6.theshire.local,\\*\IPC$,pgustavo,0x4e13b2e,0x1,2020-09-22T18:53:32.341Z,WORKSTATION6.theshire.local,\\*\C$,pgustavo,0x1,2020-09-22T18:53:32.342Z,WORKSTATION6.theshire.local,\\*\C$,pgustavo,172.18.39.5,62782.0,ProgramData\GruntHTTP.exe


In [18]:
fileAccessedDf = (
# (
df[['@timestamp','Hostname','ShareName','SubjectUserName','SubjectLogonId','IpAddress','IpPort','RelativeTargetName']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 5145)
    & (df['AccessMask'] == '0x2')
]
)

fileAccessedDf['Filename'] = fileAccessedDf['RelativeTargetName'].str.split('\\').str[-1]

fileCreateDf = (
# (
df[['@timestamp','Hostname','Image','TargetFilename']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 11)
    & (df['Image'].str.lower() == 'system')
]
)

fileCreateDf['Filename'] = fileCreateDf['TargetFilename'].str.split('\\').str[-1]

(
pd.merge(fileAccessedDf, fileCreateDf,
    on = 'Filename', how = 'inner')
)

,@timestamp_x,Hostname_x,ShareName,SubjectUserName,SubjectLogonId,IpAddress,IpPort,RelativeTargetName,Filename,@timestamp_y,Hostname_y,Image,TargetFilename
0,2020-09-22T18:53:32.342Z,WORKSTATION6.theshire.local,\\*\C$,pgustavo,0x4e13b2e,172.18.39.5,62782.0,ProgramData\GruntHTTP.exe,GruntHTTP.exe,2020-09-22T18:53:33.407Z,WORKSTATION6.theshire.local,System,C:\ProgramData\GruntHTTP.exe
